# Basic working file for move files to and from linux and between buckets in google cloud.

In [ ]:
from imports import *

Skjema Data

In [ ]:
aar = '2023'


fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/dynadata_5823.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
# skjema = table.to_pandas()

# Create a PyArrow array filled with the value '2022' for each row
# Create the 'aar' column as a categorical (dictionary-encoded) string
num_rows = table.num_rows
year_array = pa.array([aar] * num_rows, type=pa.string())
year_array = pa.DictionaryArray.from_arrays(pa.array(range(num_rows), type=pa.int32()), year_array)

# Add the new column to your existing table
table_with_aar = table.append_column('aar', year_array)

columns = {
    "ENHETS_ID": "id",
    "RAD_NR": "radnr",
    "FELT_ID": "feltnavn",
    "FELT_VERDI": "feltverdi",
    "SKJEMA": "skjema",
    "LOPENR": "lopenr",
    "AKTIV": "aktiv",
    "DELREG_NR": "delreg",
}

# Get the current schema of the table with the 'aar' column
schema = table_with_aar.schema

# Create a list to hold the new fields with renamed column names
new_fields = []

# Loop through the current schema and rename the columns based on the mapping
for field in schema:
    new_name = columns.get(field.name, field.name)  # Rename if in mapping, else keep the original name
    new_fields.append(pa.field(new_name, field.type))

# Create a new schema with the renamed fields
new_schema = pa.schema(new_fields)

# Apply the new schema to the table with 'aar' column by reconstructing it
table_renamed = pa.Table.from_arrays(table_with_aar.columns, schema=new_schema)

# Verify the result
print(table_renamed.schema)

pq.write_table(
    table_renamed,
    f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={aar}/skjema=RA-1403/skjemadata_data_0.parquet",
    filesystem=fs
)

Statistiskfilene

In [ ]:
year = 2022

In [ ]:



## Bedrift NR


fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/statistikkfil_bedrifter_nr_{year}.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table1 = dataset.read()


pq.write_table(
    table1,
    f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_nr.parquet",
    filesystem=fs
)

# pq.write_table(
#     table1,
#     f"gs://ssb-prod-noeku-data-delt/statistikkfiler/g{year}/statistikkfil_bedrifter_nr.parquet",
#     filesystem=fs
# )

del table1, dataset

In [ ]:
## Bedrift Pub

fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/statistikkfil_bedrifter_pub_{year}.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table2 = dataset.read()

table2.columns = table2.columns.str.lower()


pq.write_table(
    table2,
    f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet",
    filesystem=fs
)

# pq.write_table(
#     table2,
#     f"gs://ssb-prod-noeku-data-delt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet",
#     filesystem=fs
# )

del table2, dataset

In [ ]:
## Foretak NR

fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/statistikkfil_foretak_nr_{year}.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table3 = dataset.read()


pq.write_table(
    table3,
    f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_nr.parquet",
    filesystem=fs
)

# pq.write_table(
#     table3,
#     f"gs://ssb-prod-noeku-data-delt/statistikkfiler/g{year}/statistikkfil_foretak_nr.parquet",
#     filesystem=fs
# )

del table3, dataset

In [ ]:
#### Foretak PUB


fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/statistikkfil_foretak_pub_{year}.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table4 = dataset.read()
foretak_pub = table4.to_pandas()

foretak_pub.columns = foretak_pub.columns.str.lower()


# pq.write_table(
#     table4,
#     f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet",
#     filesystem=fs
# )


foretak_pub.to_parquet(
    f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)


# pq.write_table(
#     table4,
#     f"gs://ssb-prod-noeku-data-delt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet",
#     filesystem=fs
# )

# del table4, dataset

In [ ]:
del table4, dataset

# print option all cols
pd.set_option("display.max_columns", None)

foretak_pub.head()

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)
print(foretak_pub.columns)


2023 skjema data

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/dynadata_6023.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
# skjema = table.to_pandas()

# Create a PyArrow array filled with the value '2022' for each row
# Create the 'aar' column as a categorical (dictionary-encoded) string
num_rows = table.num_rows
year_array = pa.array(['2023'] * num_rows, type=pa.string())
year_array = pa.DictionaryArray.from_arrays(pa.array(range(num_rows), type=pa.int32()), year_array)

# Add the new column to your existing table
table_with_aar = table.append_column('aar', year_array)

columns = {
    "ENHETS_ID": "id",
    
    "RAD_NR": "radnr",
    "FELT_ID": "feltnavn",
    "FELT_VERDI": "feltverdi",
    "SKJEMA": "skjema",
    "LOPENR": "lopenr",
    "AKTIV": "aktiv",
    "DELREG_NR": "delreg",
    "ENHETS_TYPE": "enhets_type"
}

# Get the current schema of the table with the 'aar' column
schema = table_with_aar.schema

# Create a list to hold the new fields with renamed column names
new_fields = []

# Loop through the current schema and rename the columns based on the mapping
for field in schema:
    new_name = columns.get(field.name, field.name)  # Rename if in mapping, else keep the original name
    new_fields.append(pa.field(new_name, field.type))

# Create a new schema with the renamed fields
new_schema = pa.schema(new_fields)

# Apply the new schema to the table with 'aar' column by reconstructing it
table_renamed = pa.Table.from_arrays(table_with_aar.columns, schema=new_schema)

# Verify the result
print(table_renamed.schema)

pq.write_table(
    table_renamed,
    f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar=2023/skjema=RA-0174-1/skjemadata_data_0.parquet",
    filesystem=fs
)

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-skatt-naering-data-synk-opp/test/dynadata_5923.parquet"
    )
    # if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
# skjema = table.to_pandas()

# Create a PyArrow array filled with the value '2022' for each row
# Create the 'aar' column as a categorical (dictionary-encoded) string
num_rows = table.num_rows
year_array = pa.array(['2023'] * num_rows, type=pa.string())
year_array = pa.DictionaryArray.from_arrays(pa.array(range(num_rows), type=pa.int32()), year_array)

# Add the new column to your existing table
table_with_aar = table.append_column('aar', year_array)

columns = {
    "ENHETS_ID": "id",
    
    "RAD_NR": "radnr",
    "FELT_ID": "feltnavn",
    "FELT_VERDI": "feltverdi",
    "SKJEMA": "skjema",
    "LOPENR": "lopenr",
    "AKTIV": "aktiv",
    "DELREG_NR": "delreg",
    "ENHETS_TYPE": "enhets_type"
}

# Get the current schema of the table with the 'aar' column
schema = table_with_aar.schema

# Create a list to hold the new fields with renamed column names
new_fields = []

# Loop through the current schema and rename the columns based on the mapping
for field in schema:
    new_name = columns.get(field.name, field.name)  # Rename if in mapping, else keep the original name
    new_fields.append(pa.field(new_name, field.type))

# Create a new schema with the renamed fields
new_schema = pa.schema(new_fields)

# Apply the new schema to the table with 'aar' column by reconstructing it
table_renamed = pa.Table.from_arrays(table_with_aar.columns, schema=new_schema)

# Verify the result
print(table_renamed.schema)

pq.write_table(
    table_renamed,
    f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar=2023/skjema=RA-0255-1/skjemadata_data_0.parquet",
    filesystem=fs
)

In [ ]:
current_year = '2023'
skjema_list = 'RA-0174-1'
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={current_year}/skjema={skjema_list}/*"
    )
    if f.endswith(".parquet")
]

# Assuming there's only one file in fil_path
if fil_path:
    skjema = pd.read_parquet(fil_path[0], filesystem=fs)
else:
    raise FileNotFoundError(f"No Parquet files found for year {current_year}")

In [ ]:
felt_id_values = [
    "V_ORGNR",
    "F_ADRESSE",
    "FJOR_NACE_B_T1",
    "TMP_SN2007_5",
    "B_KOMMUNENR",
    "REGTYPE",
    "B_SYSSELSETTING_SYSS",
    "TMP_NY_BDR_SYSS",
    "GJELDENDE_BDR_SYSS",
    "FJOR_SYSSEL_T1",
    "LONN_PST_AORDN",
    "GJELDENDE_LONN_KR",
    "LONN",
    "FJOR_LONN_KR_T1",
    "TMP_SNITTLONN",
    "FJOR_SNITTLONN_T1",
    "GJELDENDE_OMSETN_KR",
    "OMSETN_KR",
    "FJOR_OMSETN_KR_T1",
    "TMP_SNITTOMS",
    "FJOR_SNITTOMS_T1",
    "TMP_SALGSINT_BED",
    "TMP_FORBRUK_BED",
    "VAREKOST_BED",
    "GJELDENDE_DRIFTSK_KR",
    "DRIFTSKOST_KR",
    "FJOR_DRIFTSKOST_KR_T1",
    "NACEF_5",
    "SALGSINT",
    "FORBRUK",
    "TMP_NO_P4005",
    "TMP_AVPROS_ORGFORB",
    "ORGNR_N_1",
    "TMP_NO_OMSETN",
    "TMP_DRIFTSKOSTNAD_9010",
    "TMP_DRIFTSKOSTNAD_9910",
]

# # Filter the DataFrame for the specified field values
skjema = skjema[skjema["feltnavn"].isin(felt_id_values)]

# Pivot the DataFrame
skjema = skjema.pivot_table(
    index=["id", "radnr", "lopenr"],
    columns="feltnavn",
    values="feltverdi",
    aggfunc="first",
)
skjema = skjema.reset_index()
skjema.columns = skjema.columns.str.lower()  # Convert column names to lower case

skjema = skjema[['orgnr_n_1']]

In [ ]:
# change option to show all columns
pd.set_option("display.max_columns", None)
skjema.head()

In [ ]:
# change option to print all rows
pd.set_option("display.max_rows", None)
skjema['orgnr_n_1'].value_counts()

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g2022/statistiske_foretak_foretak.parquet"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
skjema = table.to_pandas()

In [ ]:
skjema.to_parquet(
    f"gs://ssb-prod-noeku-data-delt/statistikkfiler/g2022/statistiske_foretak_foretak.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)

In [ ]:
skjema.head()

# Transfer from old bucks to new:

In [ ]:
# Skjema data

In [ ]:
start_year = 2017
end_year = 2021
skjema_list = 'RA-1407'


for year in range(start_year, end_year + 1):
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={current_year}/skjema={skjema_list}/*",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/inndata/skjemadata/skjema={skjema_list}/aar={year}/skjemadata_data_0.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)

print("Data transfer completed for all years.")

In [ ]:
# statistikkfiler

In [ ]:
start_year = 2023
end_year = 2023


for year in range(start_year, end_year + 1):
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_nr.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_bedrifter_nr.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    del skjema
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_bedrifter_pub.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    del skjema
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_nr.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_foretak_nr.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    del skjema
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_foretak_pub.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    del skjema

print("Data transfer completed for all years.")

In [ ]:
start_year = 2022
end_year = 2022


for year in range(start_year, end_year + 1):
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistiske_foretak_bedrifter.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistiske_foretak_bedrifter.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_bedrifter_pub.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistiske_foretak_foretak.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistiske_foretak_foretak.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_foretak_pub.parquet",
        )
        # if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    if fil_path:
        skjema = pd.read_parquet(fil_path[0], filesystem=fs)
    
    destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/klargjorte-data/statistikkfiler/aar={year}/statistikkfil_foretak_pub.parquet"
    
    skjema.to_parquet(destination_path, filesystem=fs, index=False)

print("Data transfer completed for all years.")

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar=2023/skjema=RA-0174-1/skjemadata_data_0.parquet",
    )
    # if f.endswith(".parquet")
]

if fil_path:
    skjema = pd.read_parquet(fil_path[0], filesystem=fs)

destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/inndata/skjemadata/skjema=RA-0174-1/aar=2023/skjemadata_data_0.parquet"

skjema.to_parquet(destination_path, filesystem=fs, index=False)

In [ ]:
temp = pd.DataFrame()

destination_path = f"gs://ssb-strukt-naering-data-produkt-prod/naringer/utdata/temp.parquet"

temp.to_parquet(destination_path, filesystem=fs, index=False)

In [ ]:
print("skjema:", skjema, type(skjema))
